In [1]:
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
Lar=LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

spatial_arrangement(
    V::Points, 
    copEV::ChainOp, 
    copFE::ChainOp; 
    [multiproc::Bool])

Compute the arrangement on the given cellular complex 2-skeleton in 3D.

A cellular complex is arranged when the intersection of every possible pair of cell
of the complex is empty and the union of all the cells is the whole Euclidean space.
The function returns the full arranged complex as a list of vertices V and a chain of borders EV, FE, CF.

##### Additional arguments:
- `multiproc::Bool`: Runs the computation in parallel mode. Defaults to `false`.

In [2]:
function spatial_arrangement(
		V::Lar.Points, # by rows
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, 
		multiproc::Bool=false)

	# face subdivision
	rV, rcopEV, rcopFE = Lar.Arrangement.spatial_arrangement_1( V,copEV,copFE,multiproc )

	bicon_comps = Lar.Arrangement.biconnected_components(rcopEV)

	rV, rEV, rFE, rCF = Lar.Arrangement.spatial_arrangement_2(rV, rcopEV, rcopFE)
end

spatial_arrangement (generic function with 2 methods)

In [3]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

EV = sparse(Int8[
    -1  1  0  0  0  0  0  0;
    0 -1  1  0  0  0  0  0;
    0  0 -1  1  0  0  0  0;
    -1  0  0  1  0  0  0  0;
    -1  0  0  0  1  0  0  0;
    0 -1  0  0  0  1  0  0;
    0  0 -1  0  0  0  1  0;
    0  0  0 -1  0  0  0  1;
    0  0  0  0 -1  1  0  0;
    0  0  0  0  0 -1  1  0;
    0  0  0  0  0  0 -1  1;
    0  0  0  0 -1  0  0  1;
])

FE = sparse(Int8[
    1  1  1 -1  0  0  0  0  0  0  0  0;
    0  0  0  0  0  0  0  0 -1 -1 -1  1;
    -1  0  0  0  1 -1  0  0  1  0  0  0;
    0 -1  0  0  0  1 -1  0  0  1  0  0;
    0  0 -1  0  0  0  1 -1  0  0  1  0;
    0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [4]:
@btime spatial_arrangement(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  2.855 ms (58568 allocations: 2.99 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 9, 1, 2, 10, 2, 3, 11, 3  …  9, 5, 6, 10, 6, 7, 11, 7, 8, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  1, 1, -1, 1, 1, -1, 1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  2, 6, 3, 6, 3, 4, 4, 5, 5, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, 1, 1, 1, 1, 1, -1, 1, 1, -1  …  -1, -1, -1, -1, 1, -1, 1, -1, 1, 1], 6, 12), sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], Int8[-1, 1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1], 2, 6))

# Ottimizzazione

In [5]:
function spatial_arrangement0(
    V::Lar.Points, # by rows
    copEV::Lar.ChainOp,
    copFE::Lar.ChainOp, 
    multiproc::Bool=false)

# face subdivision
rV::Lar.Points, rcopEV::Lar.ChainOp, rcopFE::Lar.ChainOp = Lar.Arrangement.spatial_arrangement_1( V,copEV,copFE,multiproc )

bicon_comps = Lar.Arrangement.biconnected_components(rcopEV)

rV, rEV::Lar.ChainOp, rFE::Lar.ChainOp, rCF::SparseArrays.SparseMatrixCSC{Int8,Int} = Lar.Arrangement.spatial_arrangement_2(rV, rcopEV, rcopFE)
end

spatial_arrangement0 (generic function with 2 methods)

In [6]:
@code_warntype spatial_arrangement0(V, EV, FE, false)

MethodInstance for spatial_arrangement0(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Bool)
  from spatial_arrangement0(V::Matrix, copEV::SparseMatrixCSC{Int8, Int64}, copFE::SparseMatrixCSC{Int8, Int64}, multiproc::Bool) in Main at /Users/labaudio/Desktop/LAR-TGW-3D-master/notebooks/spatial_arrangement.ipynb:1
Arguments
  #self#::Core.Const(spatial_arrangement0)
  V::Matrix{Float64}
  copEV::SparseMatrixCSC{Int8, Int64}
  copFE::SparseMatrixCSC{Int8, Int64}
  multiproc::Bool
Locals
  @_6::Any
  @_7::Any
  rCF::SparseMatrixCSC{Int8, Int64}
  rFE::Any
  rEV::Any
  bicon_comps::Any
  rcopFE::Any
  rcopEV::Any
  rV::Any
Body::Any
1 ─ %1  = Base.getproperty(Main.Lar, :Arrangement)::Any
│   %2  = Base.getproperty(%1, :spatial_arrangement_1)::Any
│   %3  = (%2)(V, copEV, copFE, multiproc)::Any
│   %4  = Base.indexed_iterate(%3, 1)::Any
│   %5  = Core.getfield(%4, 1)::Any
│         (@_7 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_7)::

In [7]:
@btime spatial_arrangement0(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  2.863 ms (58570 allocations: 2.99 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 9, 1, 2, 10, 2, 3, 11, 3  …  9, 5, 6, 10, 6, 7, 11, 7, 8, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  1, 1, -1, 1, 1, -1, 1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  2, 6, 3, 6, 3, 4, 4, 5, 5, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, 1, 1, 1, 1, 1, -1, 1, 1, -1  …  -1, -1, -1, -1, 1, -1, 1, -1, 1, 1], 6, 12), sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], Int8[-1, 1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1], 2, 6))